In [32]:
import pandas as pd

texts = []
#with open ("C:/Users/VaishuSistas/Desktop/NLP/Assignment2/sts2016-english-with-gs-v1.0/STS2016.input.answer-answer.txt", 'rt',encoding="utf8") as myfile:  
#with open ("C:/Users/VaishuSistas/Desktop/NLP/Assignment2/sts2016-english-with-gs-v1.0/STS2016.input.headlines.txt", 'rt',encoding="utf8") as myfile:  
#with open ("C:/Users/VaishuSistas/Desktop/NLP/Assignment2/sts2016-english-with-gs-v1.0/STS2016.input.postediting.txt", 'rt',encoding="utf8") as myfile:  
#with open ("C:/Users/VaishuSistas/Desktop/NLP/Assignment2/sts2016-english-with-gs-v1.0/STS2016.input.question-question.txt", 'rt',encoding="utf8") as myfile:  


 with open ("C:/Users/VaishuSistas/Desktop/NLP/Assignment2/sts2016-english-with-gs-v1.0/STS2016.input.plagiarism.txt", 'rt',encoding="utf8") as myfile:  

    for myline in myfile:
        texts.append(myline)



sentence_list = [x for i in texts for x in i.split("\t")]

out = [sentence_list[i: i+4] for i in range(0, len(sentence_list), 4)]

df_inferSent = pd.DataFrame(out, columns = ['Sentence 1', 'Sentence 2', 'Link 1', 'Link 2'])
 

IndentationError: unexpected indent (1213404070.py, line 10)

In [33]:
model_name = 'bert-base-nli-mean-tokens'

In [34]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [35]:
from sentence_transformers import SentenceTransformer

In [36]:
model_1 = SentenceTransformer(model_name)

In [37]:
#model_2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")



# Getting the columns containing the sentences into a list

SentencesSbert_1 = df_inferSent["Sentence 1"].tolist()

SentencesSbert_2 = df_inferSent["Sentence 2"].tolist()



# Computing the embeddings for the sentences

EmbeddingsSbert_1 = model.encode(SentencesSbert_1, convert_to_tensor=True)

EmbeddingsSbert_2 = model.encode(SentencesSbert_2, convert_to_tensor=True)



# Calculting the cosine similarity of each sentences' pair, and multiple the similarity score by 5

cosine_scores_sbert_sentences = util.cos_sim(EmbeddingsSbert_1, EmbeddingsSbert_2)

In [40]:
! mkdir encoder
! curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

A subdirectory or file encoder already exists.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  146M    0  377k    0     0   311k      0  0:08:02  0:00:01  0:08:01  315k
  1  146M    1 2369k    0     0  1101k      0  0:02:16  0:00:02  0:02:14 1108k
 12  146M   12 19.0M    0     0  6192k      0  0:00:24  0:00:03  0:00:21 6217k
 24  146M   24 35.6M    0     0  8796k      0  0:00:17  0:00:04  0:00:13 8824k
 35  146M   35 52.6M    0     0  10.2M      0  0:00:14  0:00:05  0:00:09 10.5M
 47  146M   47 69.6M    0     0  11.3M      0  0:00:12  0:00:06  0:00:06 14.0M
 59  146M   59 86.6M    0     0  12.1M      0  0:00:12  0:00:07  0:00:05 16.8M
 70  146M   70  103M    0     0  12.7M      0  0:00:11  0:00:08  0:00:03 16.9M
 82 

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VaishuSistas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
 # Load model
from models import InferSent
model_version = 1 # Use model_version = 2 to use FastText
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [44]:
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model

In [45]:
# Provide Location of the Glove Folder and FastText Folder
# W2V_PATH = 'content/GloVe/glove.840B.300d.txt' if model_version == 1 else 'content/fastText/crawl-300d-2M.vec'
W2V_PATH = 'C:/Users/VaishuSistas/GloVe/glove.840B.300d.txt' if model_version == 1 else 'C:/Users/VaishuSistas/fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [46]:
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [47]:
# Load some sentences
sentences_1 = df_inferSent["Sentence 1"].tolist()
sentences_2 = df_inferSent["Sentence 2"].tolist()


In [ ]:
embeddings_1 = model.encode(df_inferSent["Sentence 1"].tolist(),True)
embeddings_2 = model.encode(df_inferSent["Sentence 2"].tolist(),True)
print('No of Sentences encoded : {0}'.format(len(embeddings_1)))
#embeddings_2

C:\Users\VaishuSistas\NLP_Assignment1\models.py:209: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


In [ ]:
embeddings_1

In [ ]:
embeddings_2

In [ ]:
 def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
inferSentScore = []
for i in range(len(embeddings_1)):
     inferSentScore.append(cosine(embeddings_1[i], embeddings_2[i]))

In [ ]:
inferSentScore

In [ ]:
inferSentScore_5 = [x* 5 for x in inferSentScore]

In [ ]:
inferSentScore_5

In [ ]:
df_inferSent.insert( loc=2 , column = "Raw Score" , value = inferSentScore)
df_inferSent.insert( loc=3 , column = "Scaled Score" , value = inferSentScore_5)

In [ ]:
del df_inferSent['Link 1']
del df_inferSent['Link 2']

In [ ]:
df_inferSent

In [ ]:
df_inferSent['Scaled Score'].max()

In [ ]:
infersent_maxScoreSentences = df_inferSent['Scaled Score'].values == df_inferSent['Scaled Score'].max()
df_inferSent.loc[infersent_maxScoreSentences]

In [ ]:
infersent_minScoreSentences = df_inferSent['Scaled Score'].values == df_inferSent['Scaled Score'].min()
df_inferSent.loc[infersent_minScoreSentences]

In [ ]:
df_inferSent.to_csv(r'STS2016.input.plagiarism.txt.csv', index = False)